`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk
WA


#### MessageGraph 역할
* MessageGraph는 **채팅 및 대화형 애플리케이션**을 구축하는 데 특화된 그래프 클래스입니다. 
    * 일반적인 StateGraph가 딕셔너리 기반의 상태를 관리하는 반면, MessageGraph는 대화의 흐름을 나타내는 **메시지 목록(list[messages])**을 핵심 상태로 사용합니다. 
    * 이를 통해 사용자와 AI 간의 상호작용을 자연스럽게 모델링하고 관리할 수 있습니다.

* MessageGraph는 다음과 같은 주요 기능을 제공합니다.
    * 메시지 기반 상태 관리: 상태를 messages라는 키를 가진 리스트로 자동 설정하여, 사용자와 AI의 대화 기록을 손쉽게 추적할 수 있습니다.
    * 자동 병합: 각 노드에서 새로운 메시지 목록을 반환하면, MessageGraph는 이를 기존 메시지 목록에 자동으로 추가(append)합니다. 
        * 이는 StateGraph에서 Annotated와 add를 사용해 수동으로 구현해야 했던 기능을 기본적으로 제공합니다.
    * 대화 중심 흐름: 대화형 에이전트의 작동 방식을 직관적으로 표현합니다. 한 노드에서 사용자 메시지를 처리하고, 다른 노드에서 AI 응답을 생성하는 등 대화의 각 단계를 명확하게 구분할 수 있습니다.

`(1) Messages State 정의`
- 이전 대화 기록을 그래프 상태에 메시지 목록으로 저장하는 것이 유용
- 그래프 상태에 Message 객체 목록을 저장하는 키(채널)를 추가하고, 이 키에 리듀서 함수를 추가 
- 리듀서 함수 선택:
    - operator.add를 사용하면: 새 메시지를 기존 목록에 단순히 추가
    - add_messages 함수를 사용하면:
        - 새 메시지는 기존 목록에 추가
        - 기존 메시지 업데이트도 올바르게 처리 (메시지 ID를 추적)
```python
class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]        
```    

`(2) RAG Chain 구성`
- 메뉴 검색을 위한 벡터저장소를 초기화 (기존 저장소를 로드)
- LangChain Runnable로 구현

In [4]:
import warnings
warnings.filterwarnings("ignore")

from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

from langchain_openai import ChatOpenAI
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

# LangGraph MessagesState라는 미리 만들어진 상태를 사용
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

from textwrap import dedent
from typing import List, Literal, Tuple
from pydantic import BaseModel, Field

import gradio as gr

from pprint import pprint


In [6]:

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# LLM 모델 
#llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
)
print(llm.model_name)

# RAG 체인 구성
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

system = """
You are a helpful assistant. Use the following context to answer the user's question:

[Context]
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}")
])

# 검색기 정의
retriever = menu_db.as_retriever(
    search_kwargs={"k": 6}
)

query = "채식주의자를 위한 메뉴를 추천해주세요."
retrieved_docs = retriever.invoke(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(vars(doc))

solar-pro
{'id': 'cc60a6e4-b579-4e27-a924-2a6e160c3810', 'metadata': {'source': '../data/restaurant_menu.txt', 'menu_number': 5, 'menu_name': '가든 샐러드'}, 'page_content': '5. 가든 샐러드\n   • 가격: ₩12,000\n   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱\n   • 설명: 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이, 당근을 더해 다양한 맛과 식감을 즐길 수 있습니다. 특제 발사믹 드레싱이 채소 본연의 맛을 살려줍니다.', 'type': 'Document'}
{'id': '2e76eb60-4531-4974-8ed5-8ed457a6cf0a', 'metadata': {'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, 'page_content': '8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.', 'type': 'Document'}
{'id': '64632976-e74c-40df-a05f-334ab5b56118', 'metadata': {'source': '../data/restaurant_menu.txt', 'menu_number': 9, 'menu_name': '치킨 콘피'}, 'page_content': '9. 치킨 콘피\n   • 가격: ₩23,000\n   • 주요 식재료: 닭다리살, 허브, 마

In [ ]:

# RAG 체인 구성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAG 체인 실행
query = "채식주의자를 위한 메뉴를 추천해주세요."
response = rag_chain.invoke(query)

# 답변 출력
print(response)

`(3) 노드(Node)`

In [7]:

class GraphState(MessagesState):
    # messages key는 기본적으로 제공 - 다른 키를 추가하고 싶을 경우 아래 주석과 같이 적용 가능 
    documents: List[Document]
    grade: float
    num_generation: int
    
# 이 함수는 사용자의 질문을 받아 문서를 검색하고 답변을 생성합니다.
def retrieve_and_respond(state: GraphState):
    print("==>1. retrieve_and_respond")
    # 'messages' 리스트의 가장 마지막 메시지를 가져옵니다.
    # state['messages'][-1]은 사용자의 마지막 질문을 가져옵니다.
    last_human_message = state['messages'][-1]
    
    # HumanMessage 객체에서 실제 질문 내용(텍스트)을 가져옵니다.
    query = last_human_message.content
    
    # retriever를 사용하여 쿼리와 관련된 문서를 벡터DB에서 검색합니다.
    retrieved_docs = retriever.invoke(query)
    
    # RAG 체인(rag_chain)을 사용하여 쿼리에 대한 최종 답변을 생성합니다.
    # 이 체인은 검색된 문서를 LLM에 전달하여 답변의 근거로 사용합니다.
    response = rag_chain.invoke(query)
    
    # 검색된 문서와 AI의 응답을 GraphState에 저장하여 반환합니다.
    # 'messages' 필드에는 새로운 AI 응답(AIMessage)이 추가됩니다.
    # 'documents' 필드에는 검색된 문서 목록이 저장됩니다.
    return {
        "messages": [AIMessage(content=response)],
        "documents": retrieved_docs
    }

#### Pydantic 모델 정의: 
* GradeResponse는 LLM의 출력 형식을 **점수(score)**와 **설명(explanation)**을 포함하는 구조로 강제합니다. 
* 이 모델을 통해 LLM은 정해진 규칙을 따르는 정형화된 JSON 응답을 생성합니다.

#### 답변 평가 함수: grade_answer 함수
* 정보 추출: state 객체에서 사용자의 **질문(-2)**과 AI의 답변(-1), 그리고 답변의 근거가 된 **문서(documents)**를 가져옵니다.
* 프롬프트 생성: LLM에게 평가 전문가 역할을 부여하는 시스템 메시지와 평가에 필요한 모든 정보를 담은 인간 메시지를 포함한 프롬프트를 만듭니다.
* 평가 체인 구성: llm.with_structured_output(schema=GradeResponse)를 사용해, LLM이 GradeResponse 모델에 맞춰 응답을 생성하도록 강제합니다.
* 평가 실행: 구성된 체인에 질문, 답변, 문서를 입력하여 실행하고, GradeResponse 객체를 얻습니다.
* 상태 업데이트: 평가 점수와 현재까지의 답변 생성 횟수를 state에 저장하여 반환합니다.

In [15]:

# Pydantic을 사용해 LLM 응답의 구조를 정의합니다.
# 이 클래스는 LLM이 반환해야 할 데이터 형식을 강제합니다.
class GradeResponse(BaseModel):
    """답변 평가 결과를 나타내는 모델입니다."""
    
    # score 필드는 0.0에서 1.0 사이의 점수를 나타냅니다.
    score: float = Field(
        #...(Ellipsis, 말줄임표)는 "이 필드는 필수값이며 기본값이 없음"을 의미합니다.
        ...,
        ge=0,  # 0보다 크거나 같음
        le=1,  # 1보다 작거나 같음
        description="0에서 1 사이의 점수, 1은 완벽한 답변을 의미"
    )
    
    # explanation 필드는 점수에 대한 설명을 담는 문자열입니다.
    explanation: str = Field(
        ...,
        description="주어진 점수에 대한 설명"
    )

# 답변 품질을 평가하는 함수
# 이 함수는 RAG 시스템의 핵심 단계 중 하나로, 생성된 답변을 자체적으로 평가합니다.
def grade_answer(state: GraphState):
    print("==>2. grade_answer")
    # LangGraph의 상태(state)에서 메시지 기록을 가져옵니다.
    messages = state['messages']
    pprint(messages)
    
    # 질문과 답변을 추출합니다.
    # -2는 사용자의 마지막 질문, -1은 AI의 마지막 답변을 의미합니다.
    question = messages[-2].content
    print('====>2. question ', type(messages[-2]))
    print(question)

    answer = messages[-1].content
    print('====>2. answer ', type(messages[-1]))
    print(answer)
    
    # 검색된 문서 목록을 가져와 프롬프트에 사용하기 위해 포맷팅합니다.
    context = format_docs(state['documents'])
    print('====>2. context ')
    print(context)

    # LLM에게 평가 전문가 역할을 부여하는 시스템 프롬프트입니다.
    grading_system = """당신은 전문 평가자입니다. 주어진 맥락을 고려하여 질문에 대한 답변의 관련성과 정확성을 평가하세요.
    1이 완벽한 점수인 0에서 1 사이의 점수를 설명과 함께 제공하세요."""

    # LLM이 평가에 사용할 입력 프롬프트 템플릿을 정의합니다.
    grading_prompt = ChatPromptTemplate.from_messages([
        ("system", grading_system),
        ("human", "[Question]\n{question}\n\n[Context]\n{context}\n\n[Answer]\n{answer}\n\n[Grade]\n")
    ])
    
    # 프롬프트와 LLM을 연결하는 체인을 만듭니다.
    # .with_structured_output() 메서드는 LLM이 GradeResponse Pydantic 모델에 맞춰 응답하도록 강제합니다.
    grading_chain = grading_prompt | llm.with_structured_output(schema=GradeResponse)
    
    # 평가 체인을 실행하고, LLM의 정형화된 응답을 받습니다.
    grade_response = grading_chain.invoke({
        "question": question,
        "context": context,
        "answer": answer
    })

    # 답변 생성 시도 횟수를 추적합니다.
    num_generation = state.get('num_generation', 0)
    num_generation += 1
    
    print('====>2. grade_response.score ')
    print(grade_response.score)
    # 평가 점수와 갱신된 생성 횟수를 상태에 저장하여 반환합니다.
    return {"grade": grade_response.score, "num_generation": num_generation}

`(4) 엣지(Edge)`
* routing 함수

In [10]:

# 이 함수는 RAG 에이전트가 다음 행동을 결정하는 '라우터' 역할을 합니다.
# 반환 값은 "retrieve_and_respond" 또는 "generate"로 고정됩니다.
# END 노드에 대한 별칭(alias)으로 "generate"로 사용합니다.
def should_retry(state: GraphState) -> Literal["retrieve_and_respond", "generate"]:
    print("==>3. should_retry 라우팅함수")
    print("----GRADTING---")
    print("Grade Score 점수 : ", state["grade"])
    print("시도횟수 = ", state["num_generation"])

    # 답변 생성 시도 횟수를 확인합니다.
    # 만약 3번 이상 시도했다면, 더 이상 재시도하지 않고 최종 답변을 생성하도록 합니다.
    if state["num_generation"] > 2:
        return "generate"
    
    # 답변의 품질 점수를 확인합니다.
    # 점수가 0.7 미만이면, 현재 답변이 충분하지 않다고 판단하고
    # 문서를 다시 검색하여 답변을 재시도하도록 합니다.
    if state["grade"] < 0.7:
        return "retrieve_and_respond"
    else:
        # 점수가 0.7 이상이면, 답변이 충분히 좋다고 판단하고
        # 최종 답변을 생성하도록 합니다.
        return "generate"

`(5) 그래프(Graph) 구성`

In [16]:
# StateGraph는 그래프의 상태를 관리하는 기본 클래스입니다.
# GraphState는 그래프가 공유하는 데이터의 구조를 정의한 사용자정의 클래스입니다.
builder = StateGraph(GraphState)

# --- Node 정의 ---
# 그래프에 두 개의 노드(처리 단계)를 추가합니다.
# "retrieve_and_respond": 문서를 검색하고 답변을 생성하는 노드입니다.
# "grade_answer": 생성된 답변의 품질을 평가하는 노드입니다.
builder.add_node("retrieve_and_respond", retrieve_and_respond)
builder.add_node("grade_answer", grade_answer)

# --- Edge(연결) 추가 ---
# 그래프의 시작과 끝, 그리고 노드 간의 흐름을 정의합니다.
# START에서 시작하여 "retrieve_and_respond" 노드로 이동합니다.
builder.add_edge(START, "retrieve_and_respond")

# "retrieve_and_respond" 노드에서 "grade_answer" 노드로 이동합니다.
builder.add_edge("retrieve_and_respond", "grade_answer")

# --- 조건부 Edge 추가 ---
# "grade_answer" 노드의 결과에 따라 다음 노드를 동적으로 결정합니다.
builder.add_conditional_edges(
    # 현재 노드: "grade_answer"
    "grade_answer",
    # 라우팅 함수: 'should_retry' 함수가 다음 노드를 결정합니다.
    should_retry,
    # 매핑: 'should_retry' 함수의 반환 값에 따라 이동할 노드를 정의합니다.
    {
        # 'should_retry'가 "retrieve_and_respond"를 반환하면, 같은 노드로 돌아가 재시도합니다.
        "retrieve_and_respond": "retrieve_and_respond",
        # 'should_retry'가 "generate"를 반환하면, 그래프 실행을 종료합니다.
        # "generate"가 'END' 노드의 별칭 역할을 합니다.
        "generate": END
    }
)

# --- 그래프 컴파일 ---
# 정의된 노드와 엣지를 기반으로 실행 가능한 그래프를 만듭니다.
# 이 단계는 그래프를 최적화하고 실행 준비를 완료합니다.
graph = builder.compile()


In [13]:
# 그래프 시각화
#display(Image(graph.get_graph().draw_mermaid_png()))

mermaid_code = graph.get_graph().draw_mermaid()
print("Mermaid Code:")
print(mermaid_code)

Mermaid Code:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve_and_respond(retrieve_and_respond)
	grade_answer(grade_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve_and_respond;
	grade_answer -. &nbsp;generate&nbsp; .-> __end__;
	grade_answer -.-> retrieve_and_respond;
	retrieve_and_respond --> grade_answer;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



* https://mermaid.live/ 에서  mermain_code 로 직접 확인한다.

* [Graph이미지](https://mermaidchart.com/play?utm_source=mermaid_live_editor&utm_medium=share#pako:eNp9kd9ugjAUxl-lOUsWTYBBUcBqvJmPsKuNhVQ5BRIopJT9M777CirRhYyb9mu_8_1OD0c41CkCg0zxJicvu3UsY50krebKLLO3TbMd1eap2b7PGWOiUK3ujQq1KvADEy7TRGHb1DKdTR3Oe7dhpP1p-4lqdivmZyga_xU57Edgyc-8sRVi21syBVr_BRHbIY9y3zbrDCUqrvGsiGMSLpypov_ypy6Gjm4zBufBdN7uUJAUBe9KTURRluxBUOEKYZWFRDvHIss18xx6VzDMeLDbdcMPhf5m7p2hn8klbi_2gTjEkpgPLPMvixSY4GWLFlSoKt5rOPaGGHSOFcbAzPbSVAyxPJm6hsvXuq6AadWZSlV3WX4VXZOa0e0Kbp5YjeHKjA_Vc91JDczzhwhgR_gCFpoHhb5Ll_4q8PzFamnBt_EsIicKaLSiy8hfhDQ6WfAzMF0nDH0aUuoFPnVdP4xOv2Qa5lY)

`(6) Graph 실행`

In [ ]:
# 초기 상태
#query = "채식주의자를 위한 메뉴를 추천해주세요."
query = "LangGraph는 무엇인가요?"
initial_state = {
    "messages": [HumanMessage(content=query)],
}

# 그래프 실행 
final_state = graph.invoke(initial_state)

# 최종 상태 출력
print("최종 상태:\n")
pprint(final_state)

In [18]:
# 최종 답변만 출력
pprint(final_state['messages'][-1].content) 

('채식주의자를 위한 메뉴로 다음과 같은 옵션을 추천드립니다:  \n'
 '\n'
 '1. **가든 샐러드 (₩12,000)**  \n'
 '   - **특징**: 유기농 채소만으로 구성된 비건(Vegan) 친화적인 샐러드입니다.  \n'
 '   - **재료**: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱.  \n'
 '   - **장점**: 동물성 성분이 전혀 없으며, 신선한 채소와 발사믹 드레싱으로 가볍게 즐기기 좋습니다.  \n'
 '\n'
 '2. **버섯 크림 수프 (₩10,000)**  \n'
 '   - **특징**: 버섯과 생크림으로 만든 크리미한 수프로, 채식주의자도 즐길 수 있습니다.  \n'
 '   - **재료**: 양송이버섯, 표고버섯, 생크림, 트러플 오일.  \n'
 '   - **주의**: 일반 채식주의(Lacto-vegetarian)는 가능하지만, 비건(Vegan) 메뉴는 아닙니다(생크림 '
 '포함).  \n'
 '\n'
 '### 추가 설명  \n'
 '- **안심 스테이크 샐러드**와 **트러플 리조또**는 각각 소고기와 치즈가 포함되어 채식주의자 메뉴에 적합하지 않습니다.  \n'
 '- **해산물 파스타**와 **치킨 콘피**는 해산물/육류가 포함되어 제외됩니다.  \n'
 '\n'
 '채식주의자 중에서도 **비건(Vegan)**을 엄격히 지키는 경우 **가든 샐러드**만 선택 가능하며, '
 '**락토-채식주의자(Lacto-vegetarian)**라면 **버섯 크림 수프**도 추가 고려 대상이 될 수 있습니다.  \n'
 '\n'
 '더 궁금한 점이 있다면 알려주세요! 😊')


## 4. Gradio 챗봇

In [24]:
from typing import List, Tuple
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

# 예시 질문 리스트
# Gradio 인터페이스에 미리 보여줄 질문들입니다. 사용자는 이 질문들을 클릭해 바로 테스트할 수 있습니다.
example_questions = [
    "채식주의자를 위한 메뉴를 추천해주세요.",
    "오늘의 스페셜 메뉴는 무엇인가요?",
    "스테이크 메뉴가 있나요?"
]

# 대답 함수 정의
# 이 함수는 Gradio의 ChatInterface에 연결되어 사용자의 질문을 처리하고 AI의 응답을 반환합니다.
def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI 모델이 이해할 수 있는 LangChain 메시지 객체 형식으로 변환합니다.
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))

        # LangGraph에 전달할 초기 상태를 구성합니다.
        # 최근 2개의 대화 기록과 현재 사용자의 질문을 포함시킵니다.
        # 이는 AI가 이전 대화의 맥락을 이해하도록 돕습니다.
        initial_state = {
            "messages": chat_history[-2:] + [HumanMessage(content=message)],
        }

        # LangGraph를 호출하여 메시지 체인을 실행하고 최종 상태를 얻습니다.
        # 이 과정에서 RAG 로직이 수행됩니다.
        final_state = graph.invoke(initial_state)
        
        # 최종 상태에서 가장 마지막에 생성된 메시지(AI의 응답)의 내용을 반환합니다.
        return final_state["messages"][-1].content
        
    except Exception as e:
        # 오류 발생 시 사용자에게 친절한 메시지를 반환하고,
        # 개발자가 디버깅할 수 있도록 콘솔에 오류를 출력합니다.
        print(f"오류가 발생했습니다: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."


# Gradio 인터페이스 생성
# 사용자와 상호작용할 UI를 만듭니다.
demo = gr.ChatInterface(
    fn=answer_invoke,  # 사용자 입력이 들어왔을 때 실행할 함수
    title="레스토랑 메뉴 AI 어시스턴트",  # UI의 제목
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",  # UI의 설명
    examples=example_questions,  # 사용자가 쉽게 시작할 수 있도록 제공되는 예시 질문들
    theme=gr.themes.Soft()  # 부드러운 색상의 UI 테마 적용
)

# Gradio 애플리케이션을 실행합니다.
# 이 함수를 호출하면 웹 서버가 시작되어 로컬에서 채팅 인터페이스에 접속할 수 있습니다.
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


==>1. retrieve_and_respond
==>2. grade_answer
[HumanMessage(content='채식주의자를 위한 메뉴를 추천해주세요.', additional_kwargs={}, response_metadata={}, id='49d2896c-1866-49f3-8c21-d64a2f2f2d5d'),
 AIMessage(content='채식주의자를 위한 메뉴로 다음과 같은 옵션을 추천드립니다:  \n\n### 1. **가든 샐러드 (₩12,000)**  \n- **주요 재료**: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱  \n- **특징**: 완전한 비건(Vegan) 메뉴로, 신선한 채소만으로 구성되어 있습니다. 발사믹 드레싱으로 가볍게 즐기기 좋습니다.  \n\n### 2. **버섯 크림 수프 (₩10,000)**  \n- **주요 재료**: 양송이버섯, 표고버섯, 생크림, 트러플 오일  \n- **주의사항**: 생크림이 포함되어 있어 **락토-채식주의자(Lacto-Vegetarian)**에게 적합합니다. 비건(Vegan)은 제외됩니다.  \n\n### 3. **트러플 리조또 (₩22,000)**  \n- **주요 재료**: 아르보리오 쌀, 블랙 트러플, 파르미지아노 치즈  \n- **주의사항**: 파르미지아노 치즈가 포함되어 있어 **락토-오보 채식주의자(Lacto-Ovo Vegetarian)**에게 적합합니다.  \n\n### ❗ **비건(Vegan) 옵션**  \n- **가든 샐러드**만 순수 비건 메뉴입니다.  \n- 다른 메뉴는 동물성 재료(치즈, 생크림 등)가 포함되어 있으니 확인이 필요합니다.  \n\n추가로 채식 전용 메뉴가 필요할 경우, 레스토랑에 미리 문의하여 재료 변경 가능성을 확인해 보세요! 😊', additional_kwargs={}, response_metadata={}, id='101f1e98-2952-47ce-b9f0-3be5b471dc67')]
====>2. question  <clas

In [25]:
# 데모 종료
demo.close()

Closing server running on port: 7860
